#  Well Log and Laboratory Data Analysis

This notebook analyzes the correlation between well log data and laboratory measurements for a well. The analysis includes data loading, cleaning, matching log data with lab samples, and statistical correlation analysis to identify relationships between log measurements and laboratory results.

# Data Loading

## Laboratory Data

First, we load the laboratory data for well from an Excel file. This data contains various measurements from core samples at specific depths.

In [ ]:
from dlisio import dlis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import traceback
import glob
import sys
sys.path.append(r"c:\Users\alghziy\Desktop\Analysis\colab\HRDH")

# Create 'imgs' folder if it doesn't exist
if not os.path.exists('imgs'):
    os.makedirs('imgs')

In [4]:
# loading lab_data from an Excel file
well_name = 'HRDH_697'
lab_data = pd.read_excel("../HRDH_LAB_DATA.xlsx", sheet_name=well_name, index_col='Depth_ft')

# convert data types to float
lab_data = lab_data.apply(pd.to_numeric, errors='coerce')


print("\nDATAFRAME STRUCTURE:")
print(f"Index: {lab_data.index.name} (shape: {lab_data.index.shape})")
print(f"Columns: {list(lab_data.columns)} (shape: {lab_data.shape},)")
# print(f"Data types:\n{lab_data.info()}\n")
print(f"Depth range: {lab_data.index.min():.2f} - {lab_data.index.max():.2f} ft")

lab_data



DATAFRAME STRUCTURE:
Index: Depth_ft (shape: (10,))
Columns: ['Sample_ID', 'XRD_Calcite', 'XRD_Dolomite', 'XRD_Ankerite', 'XRD_Siderite', 'XRD_Quartz', 'XRD_Plagioclase', 'XRD_K-Feldspar', 'XRD_Illite/Mica', 'XRD_Smectite', 'XRD_Kaolinite', 'XRD_Chlorite', 'XRD_Mixed Clay', 'XRD_Anhydrite', 'XRD_Gypsum', 'XRD_Pyrite', 'XRD_Hematite', 'XRF_Na', 'XRF_Mg', 'XRF_Al', 'XRF_Si', 'XRF_P', 'XRF_S', 'XRF_Cl', 'XRF_K', 'XRF_Ca', 'XRF_Ti', 'XRF_Mn', 'XRF_Fe', 'XRF_Zr', 'XRF_Sr', 'XRF_Ba'] (shape: (10, 32),)
Depth range: 14889.20 - 14938.50 ft


,Sample_ID,XRD_Calcite,XRD_Dolomite,XRD_Ankerite,XRD_Siderite,XRD_Quartz,XRD_Plagioclase,XRD_K-Feldspar,XRD_Illite/Mica,XRD_Smectite,...,XRF_S,XRF_Cl,XRF_K,XRF_Ca,XRF_Ti,XRF_Mn,XRF_Fe,XRF_Zr,XRF_Sr,XRF_Ba
Depth_ft,,,,,,,,,,,,,,,,,,,,,
14889.2,2,0,0,0,0,99.2,0,0.0,0.7,0,...,0.11,0.69,2.74,1.19,0.00,0,0.59,0.14,0.00,0
14893.8,3,0,0,0,0,96.1,0,0.0,3.8,0,...,0.00,1.25,5.30,1.48,0.25,0,1.14,0.08,0.00,0
14894.7,4,0,0,0,0,96.5,0,0.0,3.4,0,...,0.56,1.01,4.23,3.07,0.37,0,0.98,0.08,0.00,0
14895.3,5,0,0,0,0,95.1,0,0.0,4.8,0,...,0.11,1.14,6.24,1.19,0.70,0,1.72,0.17,0.00,0
14901.9,6,0,0,0,0,96.7,0,0.9,2.0,0,...,0.64,1.03,4.05,3.24,0.68,0,1.68,0.22,0.00,0
14903.7,7,0,0,0,0,97.4,0,0.0,2.5,0,...,0.00,0.96,4.10,0.94,0.65,0,0.80,0.16,0.00,0
14905.7,8,0,0,0,0,94.0,0,0.0,5.9,0,...,0.00,1.11,7.96,1.34,0.83,0,1.63,0.00,0.04,0
14931.5,14,0,0,0,0,92.0,0,0.0,7.9,0,...,0.15,1.33,5.96,1.68,0.86,0,1.76,0.14,0.00,0
14932.4,15,0,0,0,0,95.0,0,0.0,4.9,0,...,0.00,2.05,4.86,1.87,0.44,0,1.48,0.10,0.00,0


## Well Log Data

Next, we load the well log data from DLIS files. These files contain various log measurements recorded along the wellbore. We import custom functions from the Module.py file to help with loading and processing the DLIS files.

In [ ]:

sys.path.append(r"c:\Users\alghziy\Desktop\Analysis\colab\HRDH")
from Module import dlis_to_df, load_dlis_files_from_list


"""

"""
# List of DLIS file paths for well HRDH_697
dlis_file_paths = [
        Path(r"\\bhidhares01\GROUPS\Daleelah_Interns\2025 interns\Ziyad alghamdi\transtion_minerals\Log Data\HRDH\HRDH_697_0\8375\WL\Field\Bottom_Section\Field\Deliverables\HRDH_697_0_09JAN2012_ZDL-CN-DSL_ML_BA.dlis"),
        # 14315.0 - 19136.0
        # Path(r"\\bhidhares01\GROUPS\Daleelah_Interns\2025 interns\Ziyad alghamdi\transtion_minerals\Log Data\HRDH\HRDH_697_0\8375\WL\Field\Top_Section\Field\Deliverables\HRDH_697_0_27OCT2011_ZDL-CN-DSL-GR_ML_BA.dlis"),
        # 14516.0 - 14752.5
        # Path(r"\\bhidhares01\GROUPS\Daleelah_Interns\2025 interns\Ziyad alghamdi\transtion_minerals\Log Data\HRDH\HRDH_697_0\12\WL\Field\Deliverables\HRDH_697_0_15OCT2011_ZDL1-ZDL2-CN-DSL-GR_ML_BA.dlis"),
        # 10855.0 - 14543.8
        # Path(r"\\bhidhares01\GROUPS\Daleelah_Interns\2025 interns\Ziyad alghamdi\transtion_minerals\Log Data\HRDH\HRDH_697_0\12\WL\Field\Deliverables\HRDH_697_0_15OCT2011_ZDL1-ZDL2-CN-DSL-GR_RP_BA.dlis")
        # 11050.8 - 11467.8
]


log_df, meta = load_dlis_files_from_list(dlis_file_paths) # type: ignore

log_df    

 LOADING 1 DLIS FILES FROM LIST

 STEP 1: VALIDATING FILE PATHS
------------------------------
✅  1. HRDH_697_0_09JAN2012_ZDL-CN-DSL_ML_BA.dlis (11.9 MB)

 VALIDATION SUMMARY:
Valid files: 1
Invalid files: 0

🔧 STEP 2: LOADING DLIS FILES
------------------------------

📁 Loading file 1/1: HRDH_697_0_09JAN2012_ZDL-CN-DSL_ML_BA.dlis
 STEP 1: FILE VALIDATION
✅ File found: HRDH_697_0_09JAN2012_ZDL-CN-DSL_ML_BA.dlis
Size: 11.9 MB
Path: \\bhidhares01\GROUPS\Daleelah_Interns\2025 interns\Ziyad alghamdi\transtion_minerals\Log Data\HRDH\HRDH_697_0\8375\WL\Field\Bottom_Section\Field\Deliverables\HRDH_697_0_09JAN2012_ZDL-CN-DSL_ML_BA.dlis

 STEP 2: DLIS FILE LOADING
✅ DLIS file loaded successfully
Number of logical files: 1
Using logical file: unnamed

 STEP 3: FRAME VALIDATION
✅ Found 3 frame(s)
Frame 0: 0.500000B0
Frame 1: 0.250000B1
Frame 2: 0.250000B2
✅ Selected frame 0: 0.500000B0

 STEP 4: CHANNEL ANALYSIS
✅ Found 30 channels:
    1. TDEP            | 0.500000 ft frame 0 depth      | Dim: [

,BIT,CHT.I,CN,CNC,FRAMENO,GR,GRSL,HRD1,HRD2,K,...,SLGN,SLTM,SSD,SSN,TH,U,WTBH,ZCOR,ZDEN,ZDNC
TDEP,,,,,,,,,,,,,,,,,,,,,
14315.0,8.375,2145.426025,15.326132,15.161861,1,26.828163,27.179226,85.780708,104.951935,0.001392,...,2990.00,1329.00,4880.162598,1942.648193,6.255674,4.402928,-999.25,-0.870883,2.643326,2.632216
14315.5,8.375,2146.343018,14.691160,14.530432,2,27.401520,28.026028,100.223129,131.527252,0.001392,...,2990.00,1392.00,5318.599121,1961.818359,6.604679,4.602028,-999.25,-0.838557,2.590099,2.579279
14316.0,8.375,2146.843994,14.305127,14.146642,3,27.817524,28.849190,111.980453,152.200150,0.001392,...,2990.00,1289.00,5505.107422,1966.354248,6.713920,4.734419,-999.25,-0.815970,2.555160,2.544539
14316.5,8.375,2147.072510,14.246864,14.088723,4,28.048275,28.895748,120.573517,164.398285,0.001392,...,2990.00,1172.00,5543.049316,1948.516357,6.553044,4.867069,-999.25,-0.805525,2.532331,2.521844
14317.0,8.375,2147.041748,14.555680,14.395731,5,28.140785,28.118689,127.107819,171.573120,0.001392,...,2990.00,1187.00,5532.234375,1907.303711,6.171948,5.073438,-999.25,-0.805085,2.512135,2.501768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19134.0,-999.250,-999.250000,-999.250000,-999.250000,9639,-999.250000,-999.250000,-999.250000,-999.250000,-999.250000,...,-999.25,-999.25,-999.250000,-999.250000,-999.250000,-999.250000,-999.25,-999.250000,-999.250000,-999.250000
19134.5,-999.250,-999.250000,-999.250000,-999.250000,9640,-999.250000,-999.250000,-999.250000,-999.250000,-999.250000,...,-999.25,-999.25,-999.250000,-999.250000,-999.250000,-999.250000,-999.25,-999.250000,-999.250000,-999.250000
19135.0,-999.250,-999.250000,-999.250000,-999.250000,9641,-999.250000,-999.250000,-999.250000,-999.250000,-999.250000,...,-999.25,-999.25,-999.250000,-999.250000,-999.250000,-999.250000,-999.25,-999.250000,-999.250000,-999.250000


# Data Cleaning and Quality Assessment

In this section, we clean the log data by replacing standard null values with NaN, then assess the data quality by analyzing missing values and other data characteristics.

In [6]:

# -999.25 are usually null values inside dlis files
null_values = -999.25

# Replace -999.25 with NaN FIRST
log_df_clean = log_df.replace(null_values, np.nan)
log_df_clean


,BIT,CALZ1,CALZ2,CN,CNC,FRAMENO,GR,GRSL,HRD1,HRD12,...,TEN,TH,TTEN,U,ZCOR,ZCOR2,ZDEN,ZDEN2,ZDNC,ZDNC2
TDEP,,,,,,,,,,,,,,,,,,,,,
11050.75,12.0,12.296991,12.588511,5.158723,3.955972,1,13.050921,12.944733,332.463135,311.814545,...,-1.192048,2.087480,4551.611816,0.144946,-0.030911,-0.013277,2.932235,2.958197,2.922267,2.948585
11051.00,12.0,12.301100,12.596332,6.274786,4.953794,2,14.680331,14.561134,340.422607,317.440735,...,-1.859829,2.193671,4550.965820,0.124731,-0.031793,-0.014588,2.921719,2.947026,2.911695,2.937346
11051.25,12.0,12.309196,12.602077,7.844350,6.355440,3,17.264080,17.119183,353.893097,330.929016,...,-5.460079,2.296931,4547.399414,0.142302,-0.032364,-0.015745,2.905316,2.925977,2.895212,2.916165
11051.50,12.0,12.321783,12.604621,9.752318,8.057970,4,20.680954,20.498926,374.373383,352.368744,...,-11.170093,2.323464,4541.750000,0.214350,-0.033320,-0.016966,2.881047,2.895465,2.870833,2.885474
11051.75,12.0,12.339446,12.604426,11.800697,9.885715,5,24.487303,24.259825,401.808746,379.817566,...,-17.713905,2.277293,4535.300781,0.319927,-0.035005,-0.018134,2.849731,2.859118,2.839389,2.848937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11466.75,12.0,NaN,NaN,NaN,NaN,1665,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11467.00,12.0,NaN,NaN,NaN,NaN,1666,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11467.25,12.0,NaN,NaN,NaN,NaN,1667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Log Data Summary

Now we'll create a comprehensive summary of the log data to understand its characteristics, including depth range, curve types, data quality, and sampling information.

In [7]:

from Module import create_log_summary


# Create a comprehensive summary of the log data
summary_stats = create_log_summary(log_df)

# Return the summary_stats dictionary for use in subsequent cells
summary_stats


 COMPREHENSIVE LOG DATA SUMMARY

 DEPTH COVERAGE:
Range: 11050.8 - 11467.8 ft
Total span: 417.0 ft

 DATA STATISTICS:
Total samples: 1,669 depth points
Total curves: 39 measurement channels
Data density: 4.0 samples/ft

 MEASUREMENT TYPES:
Gamma Ray: 2 curves
Density: 2 curves
Neutron: 2 curves
Spectral: 8 curves

✅ DATA QUALITY:
Overall completeness: 100.0%
Best curve: BIT (100.0% complete)
Worst curve: BIT (100.0% complete)

 DEPTH SAMPLING:
Average interval: 0.250 ft
Min interval: 0.250 ft
Max interval: 0.250 ft


{'depth_range': (np.float64(11050.75), np.float64(11467.75)),
 'total_samples': 1669,
 'total_curves': 39,
 'avg_completeness': np.float64(100.0),
 'depth_sampling': np.float64(0.25)}

In [8]:
#Missing Data Summary for log data. Calculate missingness summary
null_pct = log_df_clean.isna().mean() * 100

# Show all columns with their percentages
missing_cols = null_pct.sort_values(ascending=False)

# Show aggregate summary statistics for many columns
print("MISSING DATA SUMMARY:")
print("=" * 50)
print(f"Log DATASET: {log_df_clean.shape[0]} samples × {log_df_clean.shape[1]} curves")

print(f"Total columns: {len(null_pct)}")
print(f"Columns with missing data: {len(missing_cols)}")
print(f"Max missing %: {null_pct.max():.1f}%")
print(f"Avg missing %: {null_pct.mean():.1f}%")

# Quick check for duplicates
# print("\nDUPLICATE DEPTH CHECK:")
# print(f"Log duplicates: {log_df_clean.index.duplicated().sum()}")
# print(f"Lab duplicates: {lab_data.index.duplicated().sum()}")

# # Show unique vs total counts
# print(f"\nLog depths - Total: {len(log_df_clean)}, Unique: {log_df_clean.index.nunique()}")
# print(f"Lab depths - Total: {len(lab_data)}, Unique: {lab_data.index.nunique()}")

print("=" * 50)


# Show detailed breakdown for all columns
# for col, pct in missing_cols.items():
#     count = log_df_clean[col].isna().sum()
#     print(f"• {col:<20}: {pct:>6.1f}% ({count} missing)")




MISSING DATA SUMMARY:
Log DATASET: 1669 samples × 39 curves
Total columns: 39
Columns with missing data: 39
Max missing %: 9.9%
Avg missing %: 5.1%


# DEPTH ANALYSIS

In [9]:
# DEPTH ANALYSIS: Compare sampling depth between log and lab data
print("=" * 50)

print("Log depths summary:")
print(f"Count: {len(log_df_clean.index)}")  
print(f"Range: {log_df_clean.index.min():.1f} - {log_df_clean.index.max():.1f} ft\n")  
log_step = np.diff(log_df_clean.index).mean()  
print(f"Depth step (mean): {log_step:.2f} ft \nMin Step: {np.diff(log_df_clean.index).min():.2f} \nMax Step: {np.diff(log_df_clean.index).max():.2f}")  


print("=" * 50)


print("Lab depths summary:")
print(f"Count: {len(lab_data.index)}") 
print(f"Range: {lab_data.index.min():.1f} - {lab_data.index.max():.1f} ft\n")  
lab_step = np.diff(lab_data.index).mean()  
print(f"Depth step (mean): {lab_step:.2f} ft \nMin Step: {np.diff(lab_data.index).min():.2f} \nMax Step: {np.diff(lab_data.index).max():.2f}")  


Log depths summary:
Count: 1669
Range: 11050.8 - 11467.8 ft

Depth step (mean): 0.25 ft 
Min Step: 0.25 
Max Step: 0.25
Lab depths summary:
Count: 10
Range: 14889.2 - 14938.5 ft

Depth step (mean): 5.48 ft 
Min Step: 0.60 
Max Step: 25.80


# Depth Overlap Analysis

We need to determine where the log and lab data overlap in terms of depth coverage. This is essential for subsequent data integration and correlation analysis.

In [10]:
#   OVERLAP ANALYSIS 
print("OVERLAP ZONE ANALYSIS:")
print("=" * 50)

# Range
print(f"Depth Ranges:")
print(f"Log Range: {log_df_clean.index.min():.1f} - {log_df_clean.index.max():.1f} ft")  
print(f"Lab Range: {lab_data.index.min():.1f} - {lab_data.index.max():.1f} ft")  

# Calculate overlap zone boundaries 
overlap_start = max(lab_data.index.min(), log_df_clean.index.min())
overlap_end = min(lab_data.index.max(), log_df_clean.index.max())
overlap_span = overlap_end - overlap_start

print(f"Overlap Range: {overlap_start:.1f} - {overlap_end:.1f} ft")
print(f"Span of overlap: {overlap_span:.1f} ft")

# Get samples in overlap zone 
log_overlap = log_df_clean[(log_df_clean.index >= overlap_start) & (log_df_clean.index <= overlap_end)]
lab_overlap = lab_data[(lab_data.index >= overlap_start) & (lab_data.index <= overlap_end)]

# 2. mean and min/max step
if len(log_overlap) > 1:
    log_step_overlap = np.diff(log_overlap.index)
    print(f"\nCORRECTED LOG STEP IN OVERLAP:")
    print(f"Step (mean): {log_step_overlap.mean():.2f} ft")
    print(f"Step Min: {log_step_overlap.min():.2f} ft")
    print(f"Step Max: {log_step_overlap.max():.2f} ft")  

if len(lab_overlap) > 1:
    lab_step_overlap = np.diff(lab_overlap.index)
    print(f"\nCORRECTED LAB STEP IN OVERLAP:")
    print(f"Step (mean): {lab_step_overlap.mean():.2f} ft")
    print(f"Step Min: {lab_step_overlap.min():.2f} ft")
    print(f"Step Max: {lab_step_overlap.max():.2f} ft")  

print(f"\nOVERLAP SUMMARY:")
print(f"Log samples in overlap: {len(lab_overlap):,}")
print(f"Lab samples in overlap: {len(lab_overlap)}")

print("=" * 50)

OVERLAP ZONE ANALYSIS:
Depth Ranges:
Log Range: 11050.8 - 11467.8 ft
Lab Range: 14889.2 - 14938.5 ft
Overlap Range: 14889.2 - 11467.8 ft
Span of overlap: -3421.5 ft

OVERLAP SUMMARY:
Log samples in overlap: 0
Lab samples in overlap: 0


# Data Integration

In this section, we match the lab data to the log data using a spatial matching approach. This creates depth-matched pairs that can be used for correlation analysis.

In [ ]:
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd
from Module import match_lab_to_log

# in ft 
tolerance_ft = 1

joined = match_lab_to_log(log_df_clean, lab_overlap, tol=tolerance_ft)

if len(joined) > 0:
    print(f"\n✅ Found {len(joined)} matches with {tolerance_ft} ft tolerance")
    
    # Show statistics
    print(f"\nMATCH STATISTICS:")
    print(f"Average distance: {joined['Distance'].mean():.3f} ft")
    print(f"Max distance: {joined['Distance'].max():.3f} ft")
    print(f"Exact matches: {(joined['Distance'] == 0).sum()}")
    print(f"Near matches: {(joined['Distance'] > 0).sum()}")
    
    # Save results for verification
    verification_df = joined[['Lab_Depth', 'Log_Depth', 'Distance']].copy()
    verification_df.to_csv('verification_matches.csv', index=False)
    print(f"\nSaved verification data to 'verification_matches.csv'")
    
else:
    print(f"\nNo matches found with {tolerance_ft} ft tolerance")
    print("Trying with larger tolerance...")
    
    # Try with diffrent ft tolerance
    joined = match_lab_to_log(log_df_clean, lab_overlap, tol=2)

joined.to_csv(f"{well_name}_joined.csv", index=False)
joined



FUNCTION START - INPUT VALIDATION:
Log DataFrame shape: (1669, 39)
Lab DataFrame shape: (0, 32)
After deduplication - Log: 1669, Lab: 0
 ERROR: Empty datasets after deduplication!

No matches found with 1 ft tolerance
Trying with larger tolerance...

FUNCTION START - INPUT VALIDATION:
Log DataFrame shape: (1669, 39)
Lab DataFrame shape: (0, 32)
After deduplication - Log: 1669, Lab: 0
 ERROR: Empty datasets after deduplication!


""


In [12]:
# COMPREHENSIVE DATA INTEGRATION ASSESSMENT
print("COMPREHENSIVE DATA INTEGRATION ASSESSMENT")
print("=" * 60)

# 1. Basic Statistics
lab_coverage = (len(joined) / len(lab_data)) * 100
log_coverage = (len(joined) / len(log_df_clean)) * 100

lab_span = lab_data.index.max() - lab_data.index.min()
log_span = log_df_clean.index.max() - log_df_clean.index.min()

matched_span = joined['Lab_Depth'].max() - joined['Lab_Depth'].min()
avg_depth_diff = abs(joined['Log_Depth'] - joined['Lab_Depth']).mean()

print(f"DATA OVERVIEW:")
print(f"Lab samples: {len(lab_data)} | Log samples: {len(log_df_clean)}")
print(f"Matched pairs: {len(joined)}")
# print(f"Data ratio: 1 lab sample per {len(log_df_clean)//len(lab_data)} log samples") not usful

# 2. Coverage Analysis
print(f"\nCOVERAGE ANALYSIS:")
print(f"Lab utilization: {lab_coverage:.1f}% ({len(joined)}/{len(lab_data)})")
print(f"Log coverage: {log_coverage:.1f}% ({len(joined)}/{len(log_df_clean)})")

# 3. Depth Range Analysis
print(f"\nDEPTH RANGE ANALYSIS:")
print(f"Lab data span: {lab_span:.1f} ft")
print(f"Log data span: {log_span:.1f} ft")
print(f"Matched (joined) span: {matched_span:.1f} ft")
print(f"Lab covers {(lab_span/log_span)*100:.1f}% of well depth")

# 4. Depth Accuracy
print(f"\nDEPTH ACCURACY:")
print(f"Average depth difference: {avg_depth_diff:.2f} ft")
print(f"Max depth difference: {abs(joined['Log_Depth'] - joined['Lab_Depth']).max():.2f} ft")
print(f"Min depth difference: {abs(joined['Log_Depth'] - joined['Lab_Depth']).min():.2f} ft")


# 6. Overlap Analysis
overlap_start = max(lab_data.index.min(), log_df_clean.index.min())
overlap_end = min(lab_data.index.max(), log_df_clean.index.max())
overlap_span = overlap_end - overlap_start
print(f"\nOVERLAP ZONE ANALYSIS:")
print(f"Overlap zone: {overlap_start:.1f} - {overlap_end:.1f} ft")

# Get samples in overlap zone - FIXED the bug
log_overlap = joined[(joined.index >= overlap_start) & (joined.index <= overlap_end)]
lab_overlap = lab_data[(lab_data.index >= overlap_start) & (lab_data.index <= overlap_end)]

print(f"Log samples in overlap: {len(log_overlap)}")
print(f"Lab samples in overlap: {len(lab_overlap)}")



print(f"\nSUMMARY: Lab utilization is {lab_coverage:.1f}%, but only {log_coverage:.1f}% log coverage.")

COMPREHENSIVE DATA INTEGRATION ASSESSMENT


KeyError: 'Lab_Depth'

# Z-Score Standardization

We standardize the data using Z-scores to make different measurements with different units comparable. Z-scores convert values to how many standard deviations they are from the mean.

In [ ]:
from scipy import stats
import pandas as pd
import numpy as np

# Calculate z-scores for joined data to be used in heatmap visualization

print("Calculating Z-Scores for Joined Data:")
print("=" * 50)

# Filter for only numeric columns, excluding depth and metadata columns
numeric_cols = joined.select_dtypes(include=np.number).columns.tolist()
numeric_cols = [col for col in numeric_cols if not any(x in col for x in ['Depth', 'FRAMENO', 'Distance', 'Match_Type'])]

# Create a new DataFrame for z-scores
joined_z = pd.DataFrame(index=joined.index)

# Track skipped columns in a list
skipped_cols = []

# Calculate z-scores for each numeric column, handling NaN values
for col in numeric_cols:
    # Get non-NaN values
    valid_data = joined[col].dropna()
    
    if len(valid_data) > 1:  # Need at least 2 values to calculate z-score
        # Calculate z-scores only for non-NaN values
        z_scores = stats.zscore(valid_data)
        
        # Create a Series with NaN values where original data had NaNs
        z_col = pd.Series(index=joined.index, dtype=float)
        z_col.loc[valid_data.index] = z_scores
        
        # Add to z-score DataFrame
        joined_z[col] = z_col
    else:
        # Add to skipped columns list instead of printing individually
        skipped_cols.append(col)

# Add depth column for reference
joined_z['Depth'] = joined['Lab_Depth']

# Print summary of results
print(f"Z-Score calculation complete for {len(numeric_cols) - len(skipped_cols)} variables")
print(f"Shape: {joined_z.shape}")
print(f"Variables standardized to μ=0, σ=1 scale")
print(f"NaN values preserved from original data")

# Print skipped columns summary
if skipped_cols:
    print(f"\nSkipped {len(skipped_cols)} columns with insufficient data points:")
    print(f"Skipped columns: {skipped_cols}")

# Display first few rows
joined_z

## Z-Score Heatmap

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Filter for measurement columns in joined_z (excluding metadata)
z_lab_vars = [col for col in joined_z.columns if col.startswith('Lab_') and 
             col not in ['Lab_Depth', 'Lab_Sample_ID', 'Depth']]

z_log_vars = [col for col in joined_z.columns if col.startswith('Log_') and 
             col not in ['Log_Depth', 'Log_FRAMENO', 'Depth']]

# Filter out columns where all values are 0 or NaN
z_lab_vars = [col for col in z_lab_vars if not (joined_z[col] == 0).all() and not joined_z[col].isna().all()]
z_log_vars = [col for col in z_log_vars if not (joined_z[col] == 0).all() and not joined_z[col].isna().all()]

# ALSO filter out constant columns
z_lab_vars = [col for col in z_lab_vars if joined_z[col].std() > 0]
z_log_vars = [col for col in z_log_vars if joined_z[col].std() > 0]

# Print variable counts
print(len(z_lab_vars), "lab variables (z-scores)")
print(len(z_log_vars), "log variables (z-scores)")

# Compute correlation matrix (log variables as rows, lab variables as columns)
z_corr_matrix = joined_z[z_lab_vars + z_log_vars].corr().loc[z_log_vars, z_lab_vars]

# Create a larger figure with additional left margin for y-labels
plt.figure(figsize=(24, 24))

# Add more space on the left side of the plot
plt.subplots_adjust(left=0.2)  # Increase left margin to make room for y-labels

# Plot the heatmap with improved label visibility
hm = sns.heatmap(z_corr_matrix, 
            annot=True,                  # Show correlation values
            cmap='RdYlGn',               # Use a diverging colormap
            vmin=-1, vmax=1,             # Fixed scale for correlation values
            linewidths=0.5,              # Add line separators
            fmt='.2f',                   # Format as 2 decimal places
            annot_kws={'size': 20},      # Annotation font size
            cbar_kws={"shrink": 0.8})    # Adjust colorbar

# Labeling and styling
plt.title('Z-Score Correlation Between Log Measurements (y-axis) and Lab Measurements (x-axis)', fontsize=20)

# Improve x-axis label formatting
plt.xticks(rotation=45, ha='right', fontsize=20)

# Improve y-axis label formatting - make them more visible
plt.yticks(fontsize=20, rotation=0)  # Horizontal labels (0 degrees rotation)

# Add clearer axis labels
plt.xlabel('Lab Measurements (Z-Scores)', fontsize=25)
plt.ylabel('Log Measurements (Z-Scores)', fontsize=25)

# Apply tight_layout AFTER subplots_adjust to prevent overrides
plt.tight_layout()

# Move the y-label to be more visible
plt.gcf().axes[0].yaxis.set_label_coords(-0.15, 0.5)  # Adjust position as needed

# Save with bigger bbox_inches to ensure nothing gets cut off
plt.savefig('imgs/HRDH__zscore_correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

# Add a summary of the strongest correlations
print("\nSTRONGEST Z-SCORE CORRELATIONS:")

# Get the top 5 positive correlations
top_pos = z_corr_matrix.unstack().sort_values(ascending=False)[~z_corr_matrix.unstack().index.duplicated(keep='first')][:5] # type: ignore
print("Top positive correlations:")
for (log_var, lab_var), corr in top_pos.items(): # type: ignore
    print(f"{log_var} ↔ {lab_var}: r = {corr:.3f}")

# Get the top 5 negative correlations
top_neg = z_corr_matrix.unstack().sort_values()[~z_corr_matrix.unstack().index.duplicated(keep='first')][:5] # type: ignore
print("\nTop negative correlations:")
for (log_var, lab_var), corr in top_neg.items(): # type: ignore
    print(f"{log_var} ↔ {lab_var}: r = {corr:.3f}")

## Z-Score enhanced Heatmap

In [ ]:
# Z-SCORE ENHANCED CORRELATION ANALYSIS
from scipy.stats import pearsonr, spearmanr
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr
from Module import create_zscore_enhanced_heatmap, enhanced_zscore_correlation_analysis
warnings.filterwarnings('ignore')


# Get z-score variables
z_lab_vars = [col for col in joined_z.columns if col.startswith('Lab_') and 
            col not in ['Lab_Depth', 'Lab_Sample_ID', 'Depth']]

z_log_vars = [col for col in joined_z.columns if col.startswith('Log_') and 
            col not in ['Log_Depth', 'Log_FRAMENO', 'Depth']]

# Filter out columns where all values are 0 or NaN
z_lab_vars = [col for col in z_lab_vars if not (joined_z[col] == 0).all() and not joined_z[col].isna().all()]
z_log_vars = [col for col in z_log_vars if not (joined_z[col] == 0).all() and not joined_z[col].isna().all()]

# ALSO filter out constant columns
z_lab_vars = [col for col in z_lab_vars if joined_z[col].std() > 0]
z_log_vars = [col for col in z_log_vars if joined_z[col].std() > 0]


print(f"Z-score variables for analysis: {len(z_log_vars)} log vars, {len(z_lab_vars)} lab vars")

# Run enhanced correlation analysis on z-scores
z_correlation_results = enhanced_zscore_correlation_analysis(joined_z, z_log_vars, z_lab_vars)

# Create enhanced correlation heatmap for z-scores
create_zscore_enhanced_heatmap(z_correlation_results)

# Visualization of Results

## Match Distances Plot

This visualization shows the distribution of depth differences between matched log and lab data points, which helps assess the quality of our spatial matching.

In [ ]:
# Visualize the quality of matches
plt.figure(figsize=(12, 6))

# Left plot: Distance histogram
plt.subplot(1, 2, 1)
sns.histplot(joined['Distance'], bins=20, kde=True)
plt.xlabel('Match Distance (ft)')
plt.ylabel('Frequency')
plt.title('Distribution of Match Distances')
plt.grid(True)

# Right plot: Match types
plt.subplot(1, 2, 2)
match_counts = joined['Match_Type'].value_counts()
plt.pie(match_counts, labels=match_counts.index, autopct='%1.1f%%', 
        colors=['#66b3ff', '#99ff99'], startangle=90)
plt.axis('equal')
plt.title('Match Type Distribution')

plt.tight_layout()
plt.savefig('imgs/match_quality.png', dpi=300, bbox_inches='tight')
plt.show()

## Correlation Heatmap

This heatmap visualizes the strength of correlations between log and lab measurements, with color intensity indicating correlation strength and direction.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


# Filter for all lab measurement columns (excluding non-measurement columns)
lab_vars = [col for col in joined.columns if col.startswith('Lab_') and 
           col not in ['Lab_Depth', 'Lab_Sample_ID']]

# Filter for all log measurement columns (excluding non-measurement or auxiliary columns)
# Comment out the original code that selects all log variables
log_vars = [col for col in joined.columns if col.startswith('Log_') and 
           col not in ['Log_Depth', 'Log_FRAMENO']]

# Only include specific log measurements
# specific_logs = ['CN', 'CNC', 'GR', 'GRSL', 'HRD1', 'HRD2', 'K', 'KTH', 'LSN', 
#                 'PE', 'QPKS', 'SFT2', 'SHR', 'SLTM', 'ZDNC']
# log_vars = [f'Log_{log}' for log in specific_logs if f'Log_{log}' in joined.columns]

# Filter out columns where all values are 0 or NaN
lab_vars = [col for col in lab_vars if not (joined[col] == 0).all() and not joined[col].isna().all()]
log_vars = [col for col in log_vars if not (joined[col] == 0).all() and not joined[col].isna().all()]

# ALSO filter out constant columns
lab_vars = [col for col in lab_vars if joined[col].std() > 0]
log_vars = [col for col in log_vars if joined[col].std() > 0]  # FIXED: Now filtering log_vars properly

# Print variable counts for verification
print(f"{len(lab_vars)} lab variables")
print(f"{len(log_vars)} log variables")

# Compute correlation matrix (log variables as rows, lab variables as columns)
corr_matrix = joined[lab_vars + log_vars].corr().loc[log_vars, lab_vars]

# Create a larger figure for better readability
plt.figure(figsize=(24, 24))

# Plot the heatmap with rotated x-axis labels
sns.heatmap(corr_matrix, 
            annot=True,                  # Show correlation values
            cmap='RdYlGn',               # Use a diverging colormap
            vmin=-1, vmax=1,             # Fixed scale for correlation values
            linewidths=0.5,              # Add line separators
            fmt='.2f',                   # Format as 2 decimal places
            annot_kws={'size': 20},       # Smaller annotation font size
            cbar_kws={"shrink": 0.8})    # Adjust colorbar
            

#  labeling and styling
plt.title('Correlation Between Log Measurements (y-axis) and Lab Measurements (x-axis)', fontsize=20)
plt.xticks(rotation=45, ha='right', fontsize=20)      
plt.yticks(rotation=0, fontsize=20)                   
# Add clearer axis labels
plt.xlabel('Lab Measurements', fontsize=25, )
plt.ylabel('Log Measurements', fontsize=25, )

plt.tight_layout()
plt.savefig(f'imgs/{well_name}_correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## Enhanced Correlation Heatmap

This enhanced heatmap includes statistical significance information, highlighting only correlations that pass our significance threshold (p ≤ 0.05).

In [ ]:
# ENHANCED CORRELATION ANALYSIS (non-Z-score version)
from scipy.stats import pearsonr, spearmanr
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Module import create_enhanced_correlation_heatmap, enhanced_correlation_analysis
warnings.filterwarnings('ignore')

# Get regular variables (not z-scores)
lab_vars = [col for col in joined.columns if col.startswith('Lab_') and 
           col not in ['Lab_Depth', 'Lab_Sample_ID']]

log_vars = [col for col in joined.columns if col.startswith('Log_') and 
           col not in ['Log_Depth', 'Log_FRAMENO']]

# Filter out columns where all values are 0 or NaN
lab_vars = [col for col in lab_vars if not (joined[col] == 0).all() and not joined[col].isna().all()]
log_vars = [col for col in log_vars if not (joined[col] == 0).all() and not joined[col].isna().all()]

# ALSO filter out constant columns
lab_vars = [col for col in lab_vars if joined[col].std() > 0]
log_vars = [col for col in log_vars if joined[col].std() > 0]

print(f"Variables for analysis: {len(log_vars)} log vars, {len(lab_vars)} lab vars")

# Run enhanced correlation analysis on regular (non-z-score) data
correlation_results = enhanced_correlation_analysis(joined, log_vars, lab_vars)

# Create enhanced correlation heatmap for regular correlations
create_enhanced_correlation_heatmap(correlation_results)

## Scatter plot

In [ ]:
# VISUALIZE SIGNIFICANT CORRELATIONS WITH SCATTER PLOTS
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from Module import visualize_significant_correlations, create_correlation_figure


# Print header
print("📊 VISUALIZING SIGNIFICANT CORRELATIONS")
print("=" * 50)

# Set parameters for correlation visualization
significance_level = 0.05  # p-value threshold
min_correlation = 0.4      # minimum correlation strength to display

# Call the function to visualize significant correlations
results = visualize_significant_correlations(
    joined, 
    correlation_results,
    significance_level=significance_level,
    min_correlation=min_correlation
)

# Create a detailed visualization for the strongest correlation
if results['positive_correlations'] or results['negative_correlations']:
    # Find the strongest correlation (positive or negative)
    all_correlations = results['positive_correlations'] + results['negative_correlations']
    if all_correlations:
        # Sort by absolute correlation value
        strongest = max(all_correlations, key=lambda x: abs(x[2]))
        log_var, lab_var, r, p, n = strongest
        
        print(f"\n🔍 DETAILED ANALYSIS OF STRONGEST CORRELATION:")
        print(f"Log Variable: {log_var}")
        print(f"Lab Variable: {lab_var}")
        print(f"Pearson's r: {r:.3f} (p-value: {p:.4f}, n={n})")
        


print("\n✅ Correlation visualization complete!")

## Network

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
from Module import create_correlation_network



# Use the function with your correlation results
create_correlation_network(correlation_results, min_correlation=0.75, max_connections=30)

## Distribution of Pearson's Correlation Coefficients

This visualization shows the distribution of all Pearson correlation coefficients (r-values) between log and lab measurements, helping us understand the overall relationship patterns.

In [ ]:
# Extract all unique correlation coefficients
vals = z_corr_matrix.values
mask = np.triu(np.ones_like(vals), k=1).astype(bool)
r_vals = vals[mask]

# Compute comprehensive statistics about the correlation distribution
mean_r = np.mean(r_vals)
median_r = np.median(r_vals)
min_r = np.min(r_vals)
max_r = np.max(r_vals)
std_r = np.std(r_vals)

# Count correlations by strength category
strong_pos_count = np.sum(r_vals > 0.6)
strong_neg_count = np.sum(r_vals < -0.6)
moderate_count = np.sum((np.abs(r_vals) > 0.3) & (np.abs(r_vals) <= 0.6))
weak_count = np.sum(np.abs(r_vals) <= 0.3)

# Create descriptive text for the plot
stats_text = (
    f"Statistics:\n"
    f"Mean: {mean_r:.2f}\n"
    f"Median: {median_r:.2f}\n"
    f"Std Dev: {std_r:.2f}\n"
    f"Range: [{min_r:.2f}, {max_r:.2f}]\n\n"
    f"Correlation Strength:\n"
    f"Strong positive (>0.6): {strong_pos_count}\n"
    f"Strong negative (<-0.6): {strong_neg_count}\n"
    f"Moderate (0.3-0.6): {moderate_count}\n"
    f"Weak (<0.3): {weak_count}"
)

# Create a better visualization
plt.figure(figsize=(10, 6))

# Main histogram with KDE
ax = plt.subplot(111)
sns.histplot(r_vals, bins=25, kde=True, color='skyblue', 
             line_kws={'linewidth': 2}, alpha=0.7)

# Add vertical lines for reference
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.7)
plt.axvline(x=mean_r, color='red', linestyle='-', label=f'Mean ({mean_r:.2f})')
plt.axvline(x=median_r, color='green', linestyle='-', label=f'Median ({median_r:.2f})')

# Shade regions by correlation strength
plt.axvspan(-1, -0.6, alpha=0.2, color='red', label='Strong negative')
plt.axvspan(-0.6, -0.3, alpha=0.1, color='orange', label='Moderate negative')
plt.axvspan(-0.3, 0.3, alpha=0.1, color='gray', label='Weak')
plt.axvspan(0.3, 0.6, alpha=0.1, color='blue', label='Moderate positive')
plt.axvspan(0.6, 1, alpha=0.2, color='green', label='Strong positive')

# Add statistics textbox
plt.text(1.02, 0.5, stats_text, transform=ax.transAxes, fontsize=9,
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Improve labels and title
plt.title("Distribution of Pearson Correlation Coefficients", fontsize=14)
plt.xlabel("Correlation Coefficient (r)", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(loc='upper left', fontsize=9)

# Adjust layout to make room for the text box
plt.tight_layout()
plt.subplots_adjust(right=0.75)

# Save high-quality figure
plt.savefig('imgs/correlation_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Print summary for notebook
print("\nCORRELATION DISTRIBUTION SUMMARY:")
print("=" * 50)
print(f"Total correlations analyzed: {len(r_vals)}")
print(f"Average correlation: {mean_r:.2f} (median: {median_r:.2f})")
print(f"Strong correlations (|r| > 0.6): {strong_pos_count + strong_neg_count} ({(strong_pos_count + strong_neg_count)/len(r_vals)*100:.1f}%)")
print(f"Positive skew: {strong_pos_count/(strong_pos_count + strong_neg_count)*100:.1f}% of strong correlations are positive")
print("=" * 50)

## mineral_composition_bar, depth_trend_plots, composite_log_plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from Module import create_mineral_composition_bar, create_depth_trend_plots, create_composite_log_plot
# Define mineral variables (XRD columns)
mineral_vars = [col for col in joined.columns if 'XRD' in col]

# Define log variables
log_vars = [col for col in joined.columns if col.startswith('Log_') and 
           col not in ['Log_Depth', 'Log_FRAMENO']]

# 1. Create mineral composition bar chart
print("\n CREATING MINERAL COMPOSITION CHART...")
if len(mineral_vars) >= 3:
    create_mineral_composition_bar(joined, mineral_vars)

# 3. Create depth trend plots for selected variables
print("\n CREATING DEPTH TREND PLOTS...")
key_vars = ['Log_GR', 'Log_ZDEN', 'Lab_XRD_Quartz']
key_vars = [var for var in key_vars if var in joined.columns]
if len(key_vars) >= 1:
    create_depth_trend_plots(joined, key_vars)


# 5. Create composite log plot
print("\n CREATING COMPOSITE LOG PLOT...")
create_composite_log_plot(joined, log_vars)

print("\n✅ ALL VISUALIZATIONS CREATED AND SAVED!")

In [ ]:
# VISUALIZATIONS - Efficient grid plots 
from Module import create_distribution_grid
print("Creating efficient distribution visualizations...")

# Split variables into logical groups for better visualization
log_measurement_vars = [v for v in log_vars if any(x in v for x in ['GR', 'ZDEN', 'CN', 'PE', 'U', 'TH', 'K'])]
lab_xrd_vars = [v for v in lab_vars if 'XRD' in v]
lab_xrf_vars = [v for v in lab_vars if 'XRF' in v]


# Create grouped visualizations with standardized naming convention
if log_measurement_vars:
    fig1 = create_distribution_grid(joined, log_measurement_vars, ncols=3)
    fig1.suptitle(f"{well_name} Log Measurements Distribution", fontsize=16, y=1.02)
    plt.savefig(f'imgs/distributions_log_{well_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

if lab_xrd_vars:
    fig2 = create_distribution_grid(joined, lab_xrd_vars, ncols=3)
    fig2.suptitle(f"{well_name} XRD Mineralogy Distribution", fontsize=16, y=1.02)
    plt.savefig(f'imgs/distributions_xrd_{well_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

if lab_xrf_vars:
    fig3 = create_distribution_grid(joined, lab_xrf_vars[:12], ncols=4)  # Limit to first 12 for readability
    fig3.suptitle(f"{well_name} XRF Elements Distribution (Top 12)", fontsize=16, y=1.02)
    plt.savefig(f'imgs/distributions_xrf_{well_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

print("✅ Distributions visualizations completed and saved with standardized naming!")

# Statistics

In [ ]:
#  Compute summary stats
stats = joined[log_vars + lab_vars].describe().T
stats['skew']    = joined[stats.index].skew()
stats['kurtosis']= joined[stats.index].kurtosis()

#  Display top 10 for review
stats[['mean','50%','std','min','max','skew','kurtosis']]